# DLC/ANIPOSE PIPELINE

Automatically converts fly videos to estimated 3D coordinates using DeepLabCut and Anipose

In [ ]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.WARNING)
logging.debug("Logging works :)")

In [ ]:
# Next two lines are for dev purposes only 
%load_ext autoreload
%autoreload 2
from pathlib import Path
from dlc import analyze_new
from preprocess import run_preprocessing
import utils

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### STEP 0: Configuration

- Add filepath to DLC network config files (e.g `dlc_networks.yml`)
- Add filepath to folder with experiment videos

In [9]:
# Set the filepath to the geontype folder containing videos
videos = r""
videos = Path(videos)


p_networks = Path(r'./common_files/dlc_networks.yml')

### STEP 1: Running DeepLabCut

This will run on a directory with fly video files and generate DLC pose estimations, outputting to the same directory.

##### Processing done:
- DeepLabCut `analyze_videos`
- DeepLabCut `filterpredictions`

In [ ]:
analyze_new(videos, p_networks)
print("Finished DLC analysis...")

### STEP 2: Preprocessing for anipose

##### Processing done:
- Fix points
- Remove columns
- Rename (GenotypeFly#-CamName)
- Convert to .h5
- Generate anipose file structure

In [ ]:
run_preprocessing(videos, p_networks)